In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pandas as pd
import tsdf

from dbpd.gait_analysis import *
from dbpd.gait_analysis_config import *


In [2]:
# Cell has the tag 'parameters'
path_to_data =  '../../../tests/data'

input_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
output_path = os.path.join(path_to_data, '3.extracted_features', 'gait')

In [3]:
config = GaitFeatureExtractionConfig()
#config.set_sampling_frequency(50)
extract_gait_features(input_path, output_path, config)

(725, 9)


In [4]:
# Cell has the tag 'parameters'
path_to_data = '../../../tests/data'
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
output_path = os.path.join(path_to_data, '4.predictions', 'gait')
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')


In [5]:
config = GaitDetectionConfig()
detect_gait(input_path, output_path, path_to_classifier_input, config)

In [6]:
# paths and files
path_to_data = '../../../tests/data/'
input_path = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
output_path = os.path.join(path_to_data, '3.extracted_features', 'gait')

In [7]:
config = ArmSwingFeatureExtractionConfig()
extract_arm_swing_features(input_path, output_path, config)

In [8]:
# paths and files
path_to_data = '../../../tests/data'
input_path = os.path.join(path_to_data, '3.extracted_features', 'gait')
output_path = os.path.join(path_to_data, '4.predictions', 'gait')
path_to_classifier_input = os.path.join(path_to_data, '0.classifiers', 'gait')

In [9]:
config = GaitDetectionConfig()
detect_gait(input_path, output_path, path_to_classifier_input, config)